(a) Download the MovieLens 100K rating dataset from
https://grouplens.org/datasets/movielens/ (the small dataset recommended for
education and development). Read the dataset, display the first few rows to understand
it, and display the count of ratings (rows) in the dataset to be sure that you download it
correctly.

(b) Implement the user-based collaborative filtering approach, using the Pearson
correlation function for computing similarities between users (4 points), and (c) the
prediction function presented in class for predicting movies scores (4 points).

(d) Design and implement a new similarity function for computing similarities between
users. Explain why this similarity function is useful for the collaborative filtering
approach. Hint: Exploiting ideas from related works are highly encouraged. 4 points

(e) Use the user-based collaborative filtering approach to produce group
recommendations. Specifically, first compute the movies recommendations for each
user in the group, and then aggregate the lists of the individual users, to produce a
single list of movies for the group. You will implement two well established aggregation
methods for producing the group recommendations.

The second aggregation method is the least misery method, where one member can act
as a veto for the rest of the group. In this case, the rating of an item for a group of user is computed as the minimum score assigned to that item in all group members
recommendations. 3 points. 

Use the MovieLens 100K rating dataset for checking the correctness of your solutions.


(f) Define a way for counting the disagreements between the users in a group and
propose a method that takes disagreements into account when computing suggestions
for the group. Implement your method and explain why it is useful when producing
group recommendations. Prepare also a short presentation (about 5 slides) to show
how your method works. 6 points